# Program to query IATE database with the REST API and save it to database
see : https://iate.europa.eu/developers

and : https://documenter.getpostman.com/view/4028985/RztoMTwn?version=latest#api-keys


see also (EU responses to covid-19) : https://eur-lex.europa.eu/homepage.html?locale=en

and : https://op.europa.eu/en/web/eudatathon/covid-19

and : https://eur-lex.europa.eu/homepage.html

http://inmyownterms.com/covid-19-glossaries-dictionaries-terminology/


# IATE API credentials (does not work)

In [2]:
# credentials
import requests
user='Cartier_LIPN'
pwd = 's8UnzDBVxFMf642m'
url = 'https://iate.europa.eu/em-api/auth/login'
payload = {'username':user,'password':pwd}
headers = {
  'Accept': 'application/json'
}

resp = requests.request("POST", url, headers=headers, data = payload)

if resp.status_code == 200:
    print(resp.json())
else:
    print(resp.status_code)

404


# A. Database / tables creation (sqlite3)

In [110]:
# create database and tables
# https://www.sqlitetutorial.net/

import sqlite3
    
conn = sqlite3.connect('iate-covid19.db')

c = conn.cursor()
# Create table concepts
langs = ['cs','da','de','en','es','fi','fr','it','nl','is','no','pt']


c.execute('''DROP TABLE IF EXISTS concepts''')
c.execute('''CREATE TABLE concepts
             (id INT PRIMARY KEY, en_lexemes TEXT, cs_def TEXT,da_def TEXT,de_def TEXT,en_def TEXT,es_def TEXT,fi_def TEXT,fr_def TEXT,it_def TEXT,nl_def TEXT,is_def TEXT,no_def TEXT,pt_def TEXT)''')
# Create table concepts_crossrefs
c.execute('''DROP TABLE IF EXISTS concepts_relations''')
c.execute('''CREATE TABLE concepts_relations
             (id_concept1 INT, id_concept2 INT, relation INT, UNIQUE(id_concept1,id_concept2, relation))''')
# Create table concepts_domains
c.execute('''DROP TABLE IF EXISTS concepts_domains''')
c.execute('''CREATE TABLE concepts_domains
             (id_concept INT, id_domain INT, UNIQUE(id_concept,id_domain))''')
# Create table domains
c.execute('''DROP TABLE IF EXISTS domains''')
c.execute('''CREATE TABLE domains
             (id TEXT PRIMARY KEY, label TEXT, level INT, parent TEXT, label_var TEXT)''')
# Create table langages
c.execute('''DROP TABLE IF EXISTS languages''')
c.execute('''CREATE TABLE languages
             (id TEXT PRIMARY KEY, label TEXT)''')
# Create table lexemes
c.execute('''DROP TABLE IF EXISTS lexemes''')
c.execute('''CREATE TABLE lexemes
             (id INT PRIMARY KEY, value TEXT, query TEXT, id_concept INT, lang TEXT, type INT, context TEXT)''')
# Create table lexemes_types
c.execute('''DROP TABLE IF EXISTS lexemes_types''')
c.execute('''CREATE TABLE lexemes_types
             (id INT, label TEXT)''')
# select examples
c.execute("SELECT * FROM concepts")
print(c.fetchone())
print(c.fetchall())
for row in c.execute('SELECT * FROM lexemes'):
    print(row)

# Save (commit) the changes
conn.commit()
conn.close()


None
[]


## Get available languages and populate database / table languages

In [111]:
# get languages
import requests

url = "https://iate.europa.eu/em-api/inventories/_languages?trans_lang=en&expand=true&limit=300&offset=0"
langs = ['cs','da','de','en','es','fi','fr','it','nl','is','no','pt']

payload = {}
headers = {
  'Accept': 'application/json'
}

resp = requests.request("GET", url, headers=headers, data = payload)

if resp.status_code == 200:
    #contents = print(response.text.encode('utf8'))
    res = []
    for item in resp.json()['items']:
        res.append((item['code'],item['name']))
        #print(item['code'], item['name'])

conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.executemany('INSERT INTO languages VALUES (?,?)', res)
conn.commit()
conn.close()

## Get term types and save them to database / table lexemes_types

In [112]:
# term types
import requests

url = "https://iate.europa.eu/em-api/inventories/_term-types?trans_lang=en&expand=true&limit=20&offset=0"
payload = {}
headers = {
    'Accept': 'application/json'
}

resp = requests.request("GET", url, headers=headers, data = payload)

if resp.status_code == 200:
    res=[]
    for item in resp.json()['items']:
        print(item['code'], item['name'])
        res.append((item['code'],item['name']))

conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.executemany('INSERT INTO lexemes_types VALUES (?,?)', res)
conn.commit()
conn.close()

def get_id_labels(table, db):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    res ={}
    query = 'SELECT id,label FROM ' + table
    #print(query)
    for row in c.execute(query):
        res[row[0]]=row[1]
        
    conn.close()
    return res


termtypes = get_id_labels('lexemes_types','iate-covid19.db')
print(termtypes)

0 abbrev
1 formula
2 phrase
3 short form
4 term
5 lookup form
6 appellation
{0: 'abbrev', 1: 'formula', 2: 'phrase', 3: 'short form', 4: 'term', 5: 'lookup form', 6: 'appellation'}


## Get domains and populate database / table domains

In [113]:
# fill domains
# get domains
import requests
import pickle

def parse_tree_domains(domain, parent, parentname, res):
        if 'lookups' in domain.keys():
            lookup = ",".join(domain['lookups'])
        else:
            lookup = ''
        if parentname =='' or parentname =='Domain code not specified':
            parentname = domain['name']
        else:
            parentname = parentname +' > ' + domain['name'] 
        #print(parentname)
        res.append((domain['code'],parentname,domain['level'],parent,lookup))
        if 'subdomains' in domain.keys():
            for d in domain['subdomains']:
                parse_tree_domains(d,domain['code'],parentname, res)
        
def query_domains():
    url = "https://iate.europa.eu/em-api/domains/_tree"
    payload = {}
    headers = {
      'Accept': 'application/json'
    }
    resp = requests.request("GET", url, headers=headers, data = payload)
    if resp.status_code == 200:
        res = []
        data = resp.json()['items']
        #print(data)
        for d in data:
            parse_tree_domains(d, '','', res)
        return res
                    
conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
res = query_domains()
#print(res)
c.executemany('INSERT INTO domains VALUES (?,?,?,?,?)', res)
conn.commit()
conn.close()


In [114]:
conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.execute("SELECT * FROM domains where level=5")
print(c.fetchone())
conn.commit()
conn.close()


def get_label(id, table, db):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    query = 'SELECT label FROM ' + table + ' where id="' + id + '"'
    #print(query)
    c.execute(query)
    res = c.fetchall()
            

    conn.commit()
    conn.close()
    return res

def get_domains(table, db):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    res ={}
    query = 'SELECT id,label FROM ' + table
    #print(query)
    for row in c.execute(query):
        res[row[0]]=row[1]
        
    conn.close()
    return res


get_label('9A4F05026F3245BD95BE7DFCE54764AC', 'domains','iate-covid19.db')
domains = get_domains('domains','iate-covid19.db')
#for d in domains.keys():
#    print(d,domains[d])

('5C182C28AE7A4E578AC83588DDCC4235', 'POLITICS > political framework > political philosophy > democracy > deliberative democracy', 5, '8CA089860329450C9C521843B6F7032B', 'discursive democracy,democratic deliberation')


In [118]:
from stop_words import get_stop_words
import re
langs = ['cs','da','de','en','es','fi','fr','it','nl','is','no','pt']
word = "the great cat"
lang = 'en'
w2 = [w for w in re.split(r"\W",word, re.I) if not(w in get_stop_words(lang))]
print(w2)
#stop_words = get_stop_words('en')
#print(stop_words)

['great', 'cat']


## Populate concepts and lexemes tables

In [122]:
import pprint,pickle, re
from stop_words import get_stop_words

pp = pprint.PrettyPrinter(indent=4)


def query_code_lexemes(query,lang='en'):
    '''
    Query IATE db for a concept code and return the lexical items in the requested language.
    
        Parameters:
                    query (str): the concept code in IATE
                    lang (str): the language code

        Returns:
                    res (list): the list of lexemes
    
    '''
    resp = requests.post('https://iate.europa.eu/em-api/entries/_search?expand=true&limit=5', 
                     json={'query':query,'search_in_fields':[8], 'source':'en'})
    if resp.status_code == 200:
        res=[]
        #print(len(resp.json()['items']), " entries")
        # just get first one
        item = resp.json()['items'][0]
        # get lang entries     
        if 'language' in item.keys() and lang in item['language']:
            for entry in item['language'][lang]['term_entries']:
                res.append(entry['term_value'])
            return res
        else:
            return []
        
    else:
        print(resp.status_code)
        return False
     
def query_code_id(query):
    '''
    Query IATE db for a concept code and return the id of the concept.
    
        Parameters:
                    query (int): the concept code in IATE

        Returns:
                    id (int): the id of the concept
    
    '''
    resp = requests.post('https://iate.europa.eu/em-api/entries/_search?expand=true&limit=5', 
                     json={'query':query,'search_in_fields':[8], 'source':'en'})
    if resp.status_code == 200:
        res=[]
        #print(len(resp.json()['items']), " entries")
        # just get first one
        item = resp.json()['items'][0]
        return item['id']        
    else:
        print(resp.status_code)
        return False
  

def query_id(query, langs):
    '''
    Query IATE db for a concept code and return all the linked information (concept, concept relations, concept domains, lexemes).
    
        Parameters:
                    query (int): the concept code in IATE
                    langs (list): the list of languages to be queried for lexemes

        Returns:
                    allitems, concept,concept_rels,concept_domains, lexemes
                    allitems (list): the list of all information retrieved
                    concept (list): the list of information for the concept (id, list of entries in English, definitions in all required languages, if exist)
                    concept_rels (list): the list of the concept relations to other concepts (id_concept1, id_concept2, relation_type)
                    concept_domains (list): the list of the domains the concept belongs to (id_concept, id_domain)
                    lexemes (list): the list of lexemes linked to the concept, for all required languages (id_lexeme,lexeme_value,id_concept, lang, lexeme_type,context)   
    '''

    resp = requests.post('https://iate.europa.eu/em-api/entries/_search?expand=true&limit=30', 
                     json={'query':query,'search_in_fields':[9], 'source':'en', 'targets':langs})
    if resp.status_code == 200:
        #print(resp.json())
        if not('items' in resp.json()):
            print(resp.json())
            return [], [],[],[], []
        print(str(len(resp.json()['items'])) + " entries")
        items = resp.json()['items']
        # structure for displaying results (debug)
        allitems=[]
        for item in resp.json()['items']:
            itemdata={}
            # get concept id
            itemdata['id'] = query # or item['id']
            #print(item['id'],item['score']) # item.keys(), 
            # crossrefs
            if 'crossrefs' in item.keys():
                #print(item['crossrefs'])
                crossrefs = [(query_code_lexemes(crossref['code']),crossref['type']) for crossref in item['crossrefs']]
                concept_rels = [(itemdata['id'], query_code_id(crossref['code']),crossref['type']) for crossref in item['crossrefs']]
                itemdata['crossrefs'] = crossrefs
                #print("Related concepts : ",crossrefs)
            else:
                concept_rels=[]
            # get domains
            if 'domains' in item.keys():
                domaincodes = [domains[dom['code']]  if dom['code'] in domains else dom['code'] for dom in item['domains']]
                concept_domains = [(itemdata['id'],dom['code']) for dom in item['domains']]
                itemdata['domains'] = domaincodes
                #print('domains:', domaincodes)
            else:
                concept_domains=[]
            # get language entries
            if 'language' in item.keys():
                # structure for saving to db/table
                lexemes=[]
                #print(item['language'].keys())
                for lang in item['language']:
                    if lang in langs:
                        # structure for display/debug purposes
                        itemdata[lang] = {}
                        itemdata[lang]['entries']=[]
                        #print('***'*20,"\nlang:",lang,"\n")#,item['language'][lang].keys()
                        # definition : we store it at the concept level
                        if 'definition' in item['language'][lang].keys():
                            #itemdata[lang]['definition']=(item['language'][lang]['definition'],item['language'][lang]['definition_references'][0]['text'])
                            itemdata[lang + '_def']= item['language'][lang]['definition'] + " - " + item['language'][lang]['definition_references'][0]['text']
                        else:
                            #itemdata[lang]['definition']=('','') 
                            itemdata[lang + '_def']=''
                        # entries
                        for entry in item['language'][lang]['term_entries']:
                                #lexemes [(id INT PRIMARY KEY, value TEXT, id_concept INT, lang TEXT, type INT, definition TEXT, context TEXT)]
                                itementry = {}
                                itementry['value']= entry['term_value']
                                # patch for mapping Czech iso code
                                if lang == 'cs':
                                    lang2 = 'cz'
                                    stopw = get_stop_words(lang2)
                                else:
                                    stopw = get_stop_words(lang)
                                itementry['query'] = " ".join([w for w in re.split(r"\W",entry['term_value'], re.I) if not(w in stopw)])

                                itementry['type']= termtypes[entry['type']]
                                if 'contexts' in entry.keys(): # just keep first context
                                    itementry['context'] = entry['contexts'][0]['context']
                                    itementry['context_ref'] = entry['contexts'][0]['reference']['text']
                                    if 'language_usage' in entry['contexts'][0].keys():
                                        itementry['language_usage'] = entry['contexts'][0]['language_usage']
                                    else:
                                        itementry['language_usage']=''
                                    if 'regional_usage' in entry['contexts'][0].keys():
                                        itementry['regional_usage'] = entry['contexts'][0]['regional_usage']
                                    else:
                                        itementry['regional_usage']=''
                                else:
                                    itementry['context'] = ''
                                    itementry['context_ref'] = ''
                                    itementry['language_usage']=''
                                    itementry['regional_usage']=''
                                    
                                lexeme = (entry['id'],itementry['value'],itementry['query'],itemdata['id'], lang, entry['type'],itementry['context'])
                                lexemes.append(lexeme)
                                itemdata[lang]['entries'].append(itementry)
                        allitems.append(itemdata)
                        # concept with english_samples and definition
                        if 'en' in itemdata.keys():
                            #print(itemdata['en'])
                            entries = ", ".join([itementry['value'] for itementry in itemdata['en']['entries']])
                            lang_defs = [itemdata[lang + '_def'] if lang + '_def' in itemdata else '' for lang in langs]
                            concept = [itemdata['id'],entries]
                            concept.extend(lang_defs)
                    
        return allitems, concept,concept_rels,concept_domains, lexemes
    else:
        print(resp.status_code)
        return [],[],[],[],[]

def save_iate_to_db(data, table, conn):
    '''
    Save data retrieved from IATE to sqlite database/table
    
        Parameters:
            data(list) : a list of tuples to save to db
            table (str): the name of the table to save to
            conn (obj): the connection to the sqlite3 db file
            
        Returns:
            True|False (bool) : True or False depending on success
    '''
    # check data is not empty
    if len(data)==0:
        log.debug('Empty data')
        return False
    # first get number of fields in data elements
    nb = len(data[0])
    placeholders = ",".join(['?' for i in range(nb)])
    print(str(nb) + ' fields')
    # generate query
    #'INSERT INTO concepts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
    query = 'INSERT INTO ' + table + ' VALUES (' + placeholders + ')'
    print(query)

    try:
        #conn=sqlite3.connect(db)
        c = conn.cursor()
        c.executemany(query,data)
        conn.commit()
        print("Total insertions into " + table + ": ",conn.total_changes)
        #conn.close()
        return True
    except sqlite3.Error as e:
        print("Error : " + str(e))
        return False

def save_to_pickle(data,pathdir):  
    outfile = open(pathdir,'wb')
    pickle.dump(data,outfile)
    outfile.close()
    
langs = ['cs','da','de','en','es','fi','fr','it','nl','is','no','pt']
id_concept = '3588006'
save_dir = './save/'
res, concept,concept_rels,concept_domains, lexemes = query_id(id_concept, langs)
print("concept fields : " + str(len(concept))) # ok
print("concept relations : " + str(len(concept_rels))) # ok
print("concept domains : " + str(len(concept_domains))) # ok
print(str(len(lexemes)) + " lexemes")
# save to pickle
save_to_pickle(concept, save_dir + id_concept + '_concept.pickle')
save_to_pickle(concept_rels, save_dir + id_concept + '_concept_relations.pickle')
save_to_pickle(concept_domains, save_dir + id_concept + '_concept_domains.pickle')
save_to_pickle(lexemes, save_dir + id_concept + '_lexemes.pickle')
        
# now populate datababase/tables
conn=sqlite3.connect('iate-covid19.db')
save_iate_to_db([concept], 'concepts', conn)
save_iate_to_db(concept_rels, 'concepts_relations', conn)
save_iate_to_db(concept_domains, 'concepts_domains', conn)
save_iate_to_db(lexemes, 'lexemes', conn)
conn.close()




1 entries
concept fields : 14
concept relations : 5
concept domains : 2
67 lexemes
14 fields
INSERT INTO concepts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
Error : UNIQUE constraint failed: concepts.id
3 fields
INSERT INTO concepts_relations VALUES (?,?,?)
Error : UNIQUE constraint failed: concepts_relations.id_concept1, concepts_relations.id_concept2, concepts_relations.relation
2 fields
INSERT INTO concepts_domains VALUES (?,?)
Error : UNIQUE constraint failed: concepts_domains.id_concept, concepts_domains.id_domain
7 fields
INSERT INTO lexemes VALUES (?,?,?,?,?,?,?)
Total insertions into lexemes:  67


# Launch process for the list of covid-19 related concepts (as of April 2020 from IATE website)

In [124]:
# load reference data
import pandas as pd

import logging
import sys
import datetime


#log = logging.getLogger()

#formatter = logging.Formatter('%(asctime)s - %(filename)s - %(levelname)s - %(message)s')

# Setup file handler
#timestamp = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
#filename=f'./logs/iate_requests.ipynb.query_id.batch.{timestamp}.log'
#fhandler  = logging.FileHandler(filename=filename)
#fhandler.setLevel(logging.DEBUG)
#fhandler.setFormatter(formatter)
#log.addHandler(fhandler)

df = pd.read_csv('./resources/iate-covid-terminology_export_en-29-04-20.csv')
df.drop(['IATE entry URL','IATE entry URL.1'], inplace=True, axis=1)
print(df.info())
ids = set(df['IATE ID'].unique())
print(len(ids))
# launch search into IATE db and save data to db
# required languages
langs = ['cs','da','de','en','es','fi','fr','it','nl','is','no','pt']
save_dir = './save/'

for id in ids :
    id_concept = str(id)
    print("Processing concept id : " + id_concept)
    res, concept,concept_rels,concept_domains, lexemes = query_id(id_concept, langs)
    print("concept fields : " + str(len(concept))) # ok
    print("concept relations : " + str(len(concept_rels))) # ok
    print("concept domains : " + str(len(concept_domains))) # ok
    print(str(len(lexemes)) + " lexemes")
    # save to pickle
    save_to_pickle(concept, save_dir + id_concept + '_concept.pickle')
    save_to_pickle(concept_rels, save_dir + id_concept + '_concept_relations.pickle')
    save_to_pickle(concept_domains, save_dir + id_concept + '_concept_domains.pickle')
    save_to_pickle(lexemes, save_dir + id_concept + '_lexemes.pickle')
            
    # now populate datababase/tables
    conn=sqlite3.connect('iate-covid19.db')
    save_iate_to_db([concept], 'concepts', conn)
    save_iate_to_db(concept_rels, 'concepts_relations', conn)
    save_iate_to_db(concept_domains, 'concepts_domains', conn)
    save_iate_to_db(lexemes, 'lexemes', conn)
    conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 684 entries, 0 to 683
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TERM           684 non-null    object
 1   IATE ID        684 non-null    int64 
 2   TERM TYPE      684 non-null    object
 3   LANGUAGE CODE  684 non-null    object
dtypes: int64(1), object(3)
memory usage: 21.5+ KB
None
270
Processing concept id : 3528195
1 entries
concept fields : 14
concept relations : 1
concept domains : 2
38 lexemes
14 fields
INSERT INTO concepts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
Total insertions into concepts:  1
3 fields
INSERT INTO concepts_relations VALUES (?,?,?)
Total insertions into concepts_relations:  2
2 fields
INSERT INTO concepts_domains VALUES (?,?)
Total insertions into concepts_domains:  4
7 fields
INSERT INTO lexemes VALUES (?,?,?,?,?,?,?)
Total insertions into lexemes:  42
Processing concept id : 1353735
1 entries
concept fields : 14
concept relations

# Now we have to change tables concepts, domains, lexemes to change primary key id as AUTOINCREMENT to enable adding entries

In [108]:
conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.execute("DELETE FROM lexemes where value='distanciation'")
#c.execute("SELECT id_domain, count(*) as cnt FROM concepts_domains group by id_domain order by cnt DESC")
print(c.fetchall())
conn.commit()
conn.close()

[]


# Sqlite3 functions

In [109]:
## create users table
query ='''
CREATE TABLE users (
  uid INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT NOT NULL,
  password TEXT NOT NULL,
  email TEXT NOT NULL,
  joining_date TEXT,
  firstname TEXT NOT NULL,
  lastname TEXT NOT NULL,
  language INTEGER NOT NULL,
  user_rights INTEGER NOT NULL DEFAULT '1')
  '''
q2 = '''SELECT * FROM sqlite_master WHERE type ="table"'''
query3 = '''
INSERT INTO users (username,password,email,joining_date,firstname,lastname, language,user_rights)
VALUES('admin', 'admin','fake@univ.fr','now','admin','admin',1,0);

'''
q4 = '''SELECT * FROM sqlite_master WHERE type ="table"'''
q5 = '''SELECT MAX(id) FROM lexemes'''
q6 = '''DELETE FROM lexemes where value="distanciation"'''
q5 = '''SELECT id, value FROM lexemes where lang="fr" and value like "distanc%"'''
q6='''SELECT distinct concepts_relations.id_concept1 as c1id, concepts_relations.id_concept2 as c2id, concepts_relations.relation as rel, concepts.en_lexemes as en_lex from concepts_relations LEFT JOIN concepts ON concepts.id=concepts_relations.id_concept2 where concepts_relations.id_concept1=36453'''
q7 = '''SELECT distinct concepts_relations.relation, c1.en_lexemes as lx1, c2.en_lexemes as lx1  from concepts_relations LEFT JOIN concepts as c1 ON c1.id=concepts_relations.id_concept1 LEFT JOIN concepts as c2 ON c2.id=concepts_relations.id_concept2 where c2.en_lexemes is not NULL'''
q8 = '''DELETE from lexemes where value="distanciation"'''
conn = sqlite3.connect('/Users/emmanuelcartier/Sites/select_covid19-project/mysql/iate-covid19.db')
c = conn.cursor()
c.execute(q8)
print(c.fetchall())
print("Total insertions into table : ",conn.total_changes)
conn.commit()
conn.close()

[]
Total insertions into table :  2


In [ ]:
import sqlite3, pickle
def save_to_pickle(data,pathdir):  
    outfile = open(pathdir,'wb')
    pickle.dump(data,outfile)
    outfile.close()
    
conn = sqlite3.connect('./db/iate-covid19.db')
c = conn.cursor()
c.execute('''SELECT distinct lexemes.value,lexemes.id_concept FROM lexemes ''')
res = c.fetchall()
conn.close()
print(len(res))
resfinal = {}
for elt in res:
    resfinal[elt[0]]= elt[1]
print(resfinal, len(resfinal))
save_to_pickle(resfinal,'lexemes_concept.pickle')


In [79]:
# dump db to sqlite statements
con = sqlite3.connect('iate-covid19.db')
with open('iate-covid19.sql', 'w') as f:
    for line in con.iterdump():
        f.write('%s\n' % line)

In [48]:
# drop users table
conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.execute("DROP TABLE users")
#print(c.fetchall())
print("Total insertions into table : ",conn.total_changes)
conn.commit()
conn.close()

Total insertions into table :  0


In [52]:
# import users.sql
with open('users.sql') as fin:
    query = fin.read()
#print(query)
conn = sqlite3.connect('iate-covid19.db')
c = conn.cursor()
c.executescript(query) # many queries
#print(c.fetchall())
print("Total insertions into table : ",conn.total_changes)
conn.commit()
conn.close()

Total insertions into table :  23
